
# Принципи на качествения код в Python

Съдържание:
- SOLID
- DRY & YAGNI
- Clean Code
- PEP8
- Pylint

Писането на код е само една част от това да си програмист. В живота на едно парче код, то по-често ще бъде прочитано, отколкото променяно. В нашият програмистки живот, по-често ще ни се налага да четем код, отколкото да пишем. Затова едно от ключовите ни умения като програмист е това да пишем качествен и четим код.

Съществуват някои широко-разпространени добри практики за писане на качествен код (а и архитектура). Ще разгледаме някои тях

## SOLID

### Single reponsibility

### Open-clozed

### Liskov-substitution

### Interface segregation

### Dependency inversion

## Clean code

### Meaningful Names

### Functions

### Comments

## PEP8

## Pylint

## Пример